# Llama API

Llama API is an easy-to-use API for chat and functions on open-source.

### Install llamaapi

In [17]:
!pip install llamaapi -q

## Import libraries and insert API token

Get your API Token at [llama-api.com](https://llama-api.com) and paste it below

In [20]:
from llamaapi import LlamaAPI
import json

# Replace 'Your_API_Token' with your actual API token
llama = LlamaAPI('Your_API_Token')

## Run a simple message

This is an example on how to call Llama API

In [19]:
# API Request JSON Cell
api_request_json = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}

# Run llama
response = llama.run(api_request_json)
print(response.json())

{'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'LLHAPPY LLAMA DAY, HUMAN! *neigh*  LLWOULD YOU LIKE TO LLHEAR ABOUT OUR LLNEW LLAMA-THEMED PRODUCTS? LLWE HAVE LLSOME LLREALLY LLCOOL LLTHINGS LLCOMING LLUP! *baa*', 'function_call': None}, 'finish_reason': 'max_token'}]}


## Run Functions

This is an example on how to run Llama Functions without streaming

In [4]:
# API Request JSON Cell
api_request_json = {
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"},
  ],
  "functions": [
    {
      "model": "llama-13b-chat",
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            },
            "days": {
              "type": "number",
              "description": "for how many days ahead you wants the forecast"
            },
            "unit": {
              "type": "string",
              "enum": ["celsius", "fahrenheit"]
            }
          }
      },
      "required": ["location", "days"]
    }
  ],
  "stream": False,
  "function_call": {"name": "get_current_weather"},
}


# Run llama
response = llama.run(api_request_json)
print(response.json())


{'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': {'location': 'Boston', 'days': 5, 'unit': 'celsius'}}}, 'finish_reason': 'function_call'}]}


## Use AI plugins as functions

In this example we will use [Plug and Plai](https://plugnplai.com) to add plugins to the functions.

Plug and Plai is a [directory of plugins](https://plugnplai.com) as well as an [open-source library](https://plugnplai.com) to load and execute plugins.

In [5]:
!pip install plugnplai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.3/756.3 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.


1. Use `plugnplai` library to load plugins from the database: https://plugnplai.com
2. Let's choose Trip.com plugin
3. Install the plugin using `Plugins.install_and_activate`
4. Get an array of the functions schemas using the built in function `plugins.functions`

In [13]:
import plugnplai as pl

allurls = pl.get_plugins()
urls = [url for url in allurls if ("klarna" in url)]
print(urls)

plugins = pl.Plugins.install_and_activate(urls)
print(json.dumps(plugins.functions, indent=2))

['https://klarna.com']
[
  {
    "name": "KlarnaProducts_productsUsingGET",
    "description": "Assistant uses the Klarna plugin to get relevant product suggestions for any shopping or product discovery purpose. Assistant will reply with the following 3 paragraphs 1) Search Results 2) Product Comparison of the Search Results 3) Followup Questions. The first paragraph contains a list of the products with their attributes listed clearly and concisely as bullet points under the product, together with a link to the product and an explanation. Links will always be returned and should be shown to the user. The second paragraph compares the results returned in a summary sentence starting with \"In summary\". Assistant comparisons consider only the most important features of the products that will help them fit the users request, and each product mention is brief, short and concise. In the third paragraph assistant always asks helpful follow-up questions and end with a question mark. When assi

### Now let's run llama with the new functions

Just add `plugins.functions` to your request

In [16]:
# API Request JSON Cell
api_request_json = {
  "messages": [
    {"role": "user", "content": "Which llama t-shirts are available on Klarna?"},
  ],
  "functions": plugins.functions,
  "stream": False,
  "function_call": "force",
  "max_tokens": 800
}

# Run llama
response = llama.run(api_request_json)
print(response.json())

{'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'KlarnaProducts_productsUsingGET', 'arguments': {'countryCode': 'US', 'q': 'llama t-shirts', 'size': 10, 'min_price': 10}}}, 'finish_reason': 'function_call'}]}


# Run LlamaAPI as Streaming (\*\*\*Not working yet, it keeps running forever\*\*\*)


Example on how to run Llama Functions with streaming

1. First change "stream" in the api_request_json to "True"
2. Use a loop to consume the stream iterator and print chunks of generated text

In [11]:

########## Streaming it not working yet, it keeps running forever ##########


# API Request JSON Cell
api_request_json = {
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"},
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            },
            "days": {
              "type": "number",
              "description": "for how many days ahead you wants the forecast"
            },
            "unit": {
              "type": "string",
              "enum": ["celsius", "fahrenheit"]
            }
          }
      },
      "required": ["location", "days"]
    }
  ],
  "stream": True,
  "function_call": {"name": "get_current_weather"},
  "model": "llama-13b-chat"
}

sequences = llama.run_jupyter(api_request_json)

# Print Response Cell
for seq in sequences:
  print(seq)

{"location":
 "Boston",
 "days":
 5,
 "unit":
 "celsius"}


KeyboardInterrupt: ignored